In [ ]:
import pandas as pd
import os
import openai
import requests
import re
import json
openai.api_key = ''

In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.1 MB/s eta 0:00:00


In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:

import requests

def get_github_folder_contents(github_folder_url):
    # Extract the owner, repository, and folder path from the given URL
    parts = github_folder_url.split('/')
    owner = parts[3]
    repo = parts[4]
    folder_path = '/'.join(parts[7:])

    # Make a request to the GitHub API to get the contents of the folder
    api_url = f'https://api.github.com/repos/{owner}/{repo}/contents/{folder_path}'
    response = requests.get(api_url)

    # Check for successful response
    if response.status_code != 200:
        raise Exception(f"Failed to fetch contents from GitHub. Status Code: {response.status_code}")

    # Parse the JSON response
    folder_contents = response.json()

    # Extract the file names from the response
    file_names = [item['name'] for item in folder_contents if item['type'] == 'file']

    return file_names

# Example usage:
github_folder_url = 'https://github.com/atreish/yukatzbot/tree/main/Text%20data/programs%20data'
file_names = get_github_folder_contents(github_folder_url)[0:]
print(file_names)


['ai.txt', 'biotech.txt', 'cyber.txt', 'data-analytics.txt', 'digital-marketing-media.txt', 'math-ma.txt', 'math-phd.txt', 'occupational-therapy.txt', 'physician-assistant.txt', 'physics.txt', 'slp.txt']


In [ ]:
text_content

'{"payload":{"allShortcutsEnabled":false,"fileTree":{"Text data/faculty data":{"items":[{"name":"ai.txt","path":"Text data/faculty data/ai.txt","contentType":"file"},{"name":"bme.txt","path":"Text data/faculty data/bme.txt","contentType":"file"},{"name":"mathematics.txt","path":"Text data/faculty data/mathematics.txt","contentType":"file"},{"name":"ot.txt","path":"Text data/faculty data/ot.txt","contentType":"file"},{"name":"pas.txt","path":"Text data/faculty data/pas.txt","contentType":"file"},{"name":"physics.txt","path":"Text data/faculty data/physics.txt","contentType":"file"},{"name":"slp.txt","path":"Text data/faculty data/slp.txt","contentType":"file"}],"totalCount":7},"Text data":{"items":[{"name":"faculty data","path":"Text data/faculty data","contentType":"directory"},{"name":"programs data","path":"Text data/programs data","contentType":"directory"},{"name":".DS_Store","path":"Text data/.DS_Store","contentType":"file"},{"name":"Abraham Oxilas.txt","path":"Text data/Abraham O

In [ ]:
import requests

# Assuming 'file_name' contains the path of the file you want to fetch
file_name = "Text data/programs data/ai.txt"
raw_file_url = f'https://raw.githubusercontent.com/atreish/yukatzbot/main/{file_name}'
response = requests.get(raw_file_url)
response.raise_for_status()
text_content = response.text

# Now 'text_content' contains the plain text content of the file
print(text_content)


Artificial Intelligence program:

In this interdisciplinary master’s degree program, you’ll design and build cutting-edge artificial intelligence technologies for a variety of applications, including finance, biotech, cybersecurity, ad-tech and health care. Working with top researchers and accomplished industry experts, you’ll bridge AI and machine learning models, such as supervised and unsupervised learning, deep learning and neural networks, and reinforcement learning with engineering best practices including problem framing, requirements gathering, UI/UX and software development. In addition, you’ll gain hands-on experience with structured and unstructured data using the latest tools, such as Python, R, SQL/NoSQL, TensorFlow, Keras and pyTorch. And you’ll be doing all of this in the heart of New York City.
The New York metropolitan area has the highest number of AI-related jobs in the country. As a graduate of Yeshiva University’s master’s degree in artificial intelligence, you’ll 

In [ ]:
unsuccessful=[]
sentences=[]
delimiter = "####"


# Process each text file
for file_name in file_names:
    # Download the text file content
    raw_file_url = f'https://raw.githubusercontent.com/atreish/yukatzbot/main/Text data/programs data/{file_name}'
    response = requests.get(raw_file_url)
    response.raise_for_status()
    text_content = response.text


    user_message_1 = f"""
Generate around 5 questions and corresponsing answers for these questions mentioning the program name mandatorily . Each question should contain his program name in the sentence.
Give answers in the senetences mentioned in the sentences. Not the list. Questions related only to the text.

"""
    messages =  [
{'role':'system',
 'content': text_content},
{'role':'user',
 'content': f"{delimiter}{user_message_1}{delimiter}"},
]
    try:
        category_and_product_respons = get_completion_from_messages(messages)
        sentences.extend(category_and_product_respons.split('\n'))

    except openai.OpenAIError as e:
        # Handle the OpenAIError exception
        unsuccessful.append(text_content)
        # Additional error handling code if needed

In [ ]:
len(sentences)

154

In [ ]:
len(sentences)

154

In [ ]:
sentences=list(filter(lambda x: x!='I can provide five questions and their corresponding answers based on the above text:',sentences))

In [ ]:
sentences=list(filter(lambda x: x!="I'm sorry, as an AI language model, I cannot provide a list of 10 questions and their corresponding answers without any context. Could you please provide more information or specify the questions you would like me to answer?",sentences))

In [ ]:
sentences=list(filter(lambda x: x!='',sentences))

In [ ]:
len(sentences)

110

In [ ]:
df=pd.DataFrame(columns=['question','answer'])

In [ ]:
for i in range(0, len(sentences), 2):
    if i + 1 < len(sentences):
        df.loc[len(df)] = {
            'question': sentences[i],
            'answer': sentences[i + 1],
        }
    else:
        df.loc[len(df)] = {
            'question': sentences[i],
            'answer': '',
        }

In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'Answer: ','',x))

In [ ]:
df['question']=df['question'].apply(lambda x: re.sub(r'\d+\.\s','',x))

In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'^\d+\.\s*','',x))

In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'^-','',x))

In [ ]:
df['question']=df['question'].apply(lambda x: re.sub(r'Q:','',x))

In [ ]:
df['answer']=df['answer'].apply(lambda x: re.sub(r'Answer:','',x))

In [ ]:
# Identify duplicate values in the 'question' column
df.drop_duplicates(subset=['question'],inplace=True)



In [ ]:
df

,question,answer
0,What are some of the applications that student...,Students in the program can work on applicati...
1,What are some of the skills and knowledge that...,Students will gain skills and knowledge in AI...
2,What are some of the job opportunities availab...,Graduates can pursue careers in research and ...
3,Can undergraduate students at Yeshiva Universi...,"Yes, through the B.A./M.S. Pathways option, u..."
4,Are there opportunities for internships and pr...,"Yes, students have the opportunity to gain in..."
5,What is the goal of the M.S. in Biotechnology ...,The goal of the M.S. in Biotechnology Managem...
6,How does the Biotech Fellows program benefit s...,The Biotech Fellows program provides students...
7,Can undergraduate students from Yeshiva Colleg...,"Yes, undergraduate students from Yeshiva Coll..."
8,Are internships available for students in the ...,"Yes, students in the Master's in Biotechnolog..."
9,Is the Master's in Biotechnology Management an...,"Yes, the Master's in Biotechnology Management..."


In [ ]:
df.to_csv('questions_answers_cleaned_programs.csv', index=False)